In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
import numpy as np
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from classifier import Classifier 

In [3]:
data = pd.read_csv('../grade_data.csv')
data.head()

,randNumCol,HW0-old,HW0-old - Max Points,HW0-old - Submission Time,HW0-old - Lateness (H:M:S),HW1,HW1 - Max Points,HW1 - Submission Time,HW1 - Lateness (H:M:S),HW2,...,Project_FA22_code - Lateness (H:M:S),Project_Fa22_ExtraCredits,Project_Fa22_ExtraCredits - Max Points,Project_Fa22_ExtraCredits - Submission Time,Project_Fa22_ExtraCredits - Lateness (H:M:S),HW0,HW0 - Max Points,HW0 - Submission Time,HW0 - Lateness (H:M:S),Total Lateness (H:M:S)
0,801925577,NaN,0.0,NaN,00:00:00,48.0,55.0,2022-09-06 08:43:22 -0700,106:44:22,48.0,...,00:01:22,4.0,15.0,2022-12-10 20:58:06 -0800,190:59:06,NaN,0.0,NaN,00:00:00,297:45:05
1,262507520,NaN,0.0,2022-08-23 09:49:04 -0700,00:00:00,40.0,55.0,2022-09-01 21:58:56 -0700,00:00:00,44.0,...,00:00:00,NaN,15.0,NaN,00:00:00,NaN,0.0,2022-08-24 14:46:39 -0700,00:00:00,00:00:00
2,853470508,NaN,0.0,NaN,00:00:00,43.0,55.0,2022-09-01 20:34:25 -0700,00:00:00,49.0,...,00:00:00,15.0,15.0,2022-12-02 19:31:27 -0800,00:00:00,NaN,0.0,2022-08-24 14:49:28 -0700,00:00:00,11:14:30
3,468287725,NaN,0.0,NaN,00:00:00,45.0,55.0,2022-09-01 21:46:46 -0700,00:00:00,51.0,...,00:00:00,4.0,15.0,2022-12-02 21:56:30 -0800,00:00:00,NaN,0.0,2022-08-30 20:43:07 -0700,00:00:00,01:12:59
4,966245481,NaN,0.0,NaN,00:00:00,52.5,55.0,2022-09-01 19:21:49 -0700,00:00:00,46.5,...,00:00:00,NaN,15.0,NaN,00:00:00,NaN,0.0,NaN,00:00:00,00:00:00


In [97]:
class Grade(Dataset):    
    def __init__(self, csv_file, max_length, transform=None):
        """
        Args:
            csv_file (string): Path to the CSV file.
            src_lang (string): Source language.
            tgt_lang (string): Target language.
            tokenizer (callable): Tokenizer function.
            max_length (int, optional): Maximum sequence length.
        """
        self.df = pd.read_csv(csv_file)  
        self.max_length = max_length
        self.transform = transform
        self.HW = [] 
        for i in range(1, 11):
            self.HW.append(f"HW{i}")
        self.data = self.df.loc[:, self.HW]
        self.data.dropna(axis=0, inplace=True)
        self.train = self.data.iloc[:80]
        self.eval = self.data.iloc[80:]

    def __len__(self):
        return len(self.train)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        rows = self.train.iloc[idx]
        input = rows[self.HW[:-1]].values
        input = np.insert(input, 0, 56)
        target = rows[self.HW[-1]]  
        return input, target

        

In [98]:
csv_file_path = "../grade_data.csv"
max_seq_length = 16

dataset = Grade(csv_file_path, max_length=max_seq_length)
print(dataset.data)
print(len(dataset))

      HW1   HW2   HW3   HW4   HW5   HW6   HW7   HW8   HW9  HW10
0    48.0  48.0  41.0  42.5  48.0  44.0  42.0  50.0  51.0  55.0
2    43.0  49.0  45.0  50.0  50.0  50.0  49.0  49.0  48.0  50.0
3    45.0  51.0  38.5  44.5  51.0  29.0  46.0  51.0  48.0  47.0
6    42.0  51.0  44.5  52.0  45.0  50.0  48.0  46.0  51.0  52.0
8    46.0  49.0  51.5  52.0  52.0  50.0  50.0  46.0  50.0  50.0
..    ...   ...   ...   ...   ...   ...   ...   ...   ...   ...
344  46.0  47.5  47.0  46.5  50.0  48.5  50.0  44.0  52.0  55.0
346  45.0  51.0  46.0  49.0  51.0  41.0  30.0  40.0  48.0  47.0
350  42.5  50.0  46.0  52.0  52.0  52.0  50.0  51.0  50.0  55.0
353  53.0  47.0  50.0  52.0  51.0  52.0  50.0  52.0  49.0  44.0
355  45.0  49.0  47.5  42.0  38.0  50.0  50.0  49.0  50.0  50.0

[95 rows x 10 columns]
80


In [31]:
input, target = dataset[0]
print(input, target)

[56.  48.  48.  41.  42.5 48.  44.  42.  50.  51. ] 55.0


In [91]:
# Parameters for the data loader
batch_size = 16
shuffle = True  

# Create a data loader
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=shuffle)
classifier = Classifier(num_layers=6, d_model=512, num_heads=8, d_ffn=256, input_vocab_size=60, output_vocab_size=60, max_seq_length=max_seq_length)
optim = torch.optim.Adam(classifier.parameters(), lr=0.0001, betas=(0.9, 0.98), eps=1e-9)

In [92]:
classifier.train()
epoch = 20

for i in range(epoch):
    print("Epoch: ", i)
    for input, target in dataloader:
        optim.zero_grad()
        
        input = input.to(torch.int64) 
        target = target.to(torch.int64) 
        
        output = classifier(input)
        output = output[:, 0, :].float()
        
        
        target = F.one_hot(target, num_classes=60).float()
        loss = F.cross_entropy(output, target)
        print(loss)

        loss.backward()
        optim.step()

Epoch:  0
tensor(4.6176, grad_fn=<DivBackward1>)
tensor(3.2284, grad_fn=<DivBackward1>)
tensor(3.4755, grad_fn=<DivBackward1>)
tensor(2.2093, grad_fn=<DivBackward1>)
tensor(2.2592, grad_fn=<DivBackward1>)
Epoch:  1
tensor(2.5384, grad_fn=<DivBackward1>)
tensor(1.6346, grad_fn=<DivBackward1>)
tensor(2.2205, grad_fn=<DivBackward1>)
tensor(1.9678, grad_fn=<DivBackward1>)
tensor(2.0560, grad_fn=<DivBackward1>)
Epoch:  2
tensor(1.8244, grad_fn=<DivBackward1>)
tensor(1.6600, grad_fn=<DivBackward1>)
tensor(2.5234, grad_fn=<DivBackward1>)
tensor(1.6779, grad_fn=<DivBackward1>)
tensor(1.9186, grad_fn=<DivBackward1>)
Epoch:  3
tensor(1.0514, grad_fn=<DivBackward1>)
tensor(1.8349, grad_fn=<DivBackward1>)
tensor(1.9747, grad_fn=<DivBackward1>)
tensor(1.9815, grad_fn=<DivBackward1>)
tensor(2.1798, grad_fn=<DivBackward1>)
Epoch:  4
tensor(1.6886, grad_fn=<DivBackward1>)
tensor(1.9341, grad_fn=<DivBackward1>)
tensor(2.0196, grad_fn=<DivBackward1>)
tensor(1.4448, grad_fn=<DivBackward1>)
tensor(1.5004,

In [93]:
model_path = 'encoder_classifier.pth'

# Save the model
torch.save(classifier.state_dict(), model_path)

In [94]:
classifier = Classifier(num_layers=6, d_model=512, num_heads=8, d_ffn=256, input_vocab_size=60, output_vocab_size=60, max_seq_length=max_seq_length)
classifier.load_state_dict(torch.load("./encoder_classifier.pth"))
classifier.eval()

Classifier(
  (encoder): Encoder(
    (embedding): WordEmbedding(
      (embedding): Embedding(60, 512)
    )
    (pe): PositionalEncoding()
    (encoder_layers): ModuleList(
      (0-5): 6 x EncoderLayer(
        (multihead_attention): MultiHeadAttention(
          (W_q): Linear(in_features=512, out_features=512, bias=True)
          (W_k): Linear(in_features=512, out_features=512, bias=True)
          (W_v): Linear(in_features=512, out_features=512, bias=True)
          (fc_out): Linear(in_features=512, out_features=512, bias=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (feedforward): FeedForward(
          (linear1): Linear(in_features=512, out_features=256, bias=True)
          (relu): ReLU()
          (dropout): Dropout(p=0.1, inplace=False)
          (linear2): Linear(in_features=256, out_features=512, bias=True)
        )
        (layer_norm1): LayerNormalization()
        (layer_norm2): LayerNormalization()
        (dropout1): Dropout(p=0.1, inplac

In [95]:
input, target = dataset[0]
print(input, target)

input = torch.from_numpy(input).to(torch.int64).unsqueeze(0)
output = classifier(input)
output = output[:, 0, :].float()
print(output.shape)
softmax_output = F.softmax(output, dim=1)
prediction = torch.argmax(softmax_output)
print(prediction)


[56.  48.  48.  41.  42.5 48.  44.  42.  50.  51. ] 55.0
torch.Size([1, 60])
tensor(55)


In [99]:
eval = dataset.eval.values
correct = 0

for i in range(len(dataset.eval)):
    input = eval[i][:9]
    target = eval[i][-1]
    input = torch.from_numpy(input).to(torch.int64).unsqueeze(0)
    output = classifier(input)
    output = output[:, 0, :].float() 
    softmax_output = F.softmax(output, dim=1)
    prediction = torch.argmax(softmax_output)
    print("Target: ", target, " Prediction: ", prediction)
    if prediction == target:
        correct += 1
print("Accuracy: ", correct / len(eval))

Target:  55.0  Prediction:  tensor(55)
Target:  55.0  Prediction:  tensor(50)
Target:  52.0  Prediction:  tensor(45)
Target:  50.0  Prediction:  tensor(55)
Target:  55.0  Prediction:  tensor(55)
Target:  50.0  Prediction:  tensor(50)
Target:  55.0  Prediction:  tensor(40)
Target:  50.0  Prediction:  tensor(50)
Target:  50.0  Prediction:  tensor(55)
Target:  55.0  Prediction:  tensor(35)
Target:  55.0  Prediction:  tensor(30)
Target:  47.0  Prediction:  tensor(55)
Target:  55.0  Prediction:  tensor(52)
Target:  44.0  Prediction:  tensor(55)
Target:  50.0  Prediction:  tensor(50)
Accuracy:  0.3333333333333333
